### Retrain after two-stage pruning.
- 用以提升經過剪枝後的模型效能
- 用於alarm, moaning, help, misc-sounds classification.

In [1]:
project_PATH = '/home/sail/sound_project/sound_ai_v2'
step_now = 'STEP/step_4'

use_model_in_step3 = "/home/sail/sound_project/sound_ai_v2/STEP/step_3/save_pt_model_s3/20240829_18/uec_secondPrun_time_2024082918_prunratio90.0/model_second_stage_prun_ratio0.9_20240829183155.pt"
second_pruned_model = use_model_in_step3

In [2]:
import os;
import sys;
sys.path.append(os.path.abspath(f'{project_PATH}'))



In [3]:
import math;
import numpy as np;
import glob;
import random;
import time;
import torch;
import torch.optim as optim;
import torch.nn as nn;

# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat
from datetime import datetime;


In [4]:
import th.resources.models_layer_cut as models

In [5]:
from SharedLibs.datestring import genDataTimeStr, getDateStr

In [6]:
seed = 1123;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [7]:
inp_len = 20150
sr = 20150
choose_class=[0,1,2,3,4,5,6]
PATH = f'{project_PATH}/STEP/data_v2.npz'

ch_n_class = 7
fcn_no_of_inputs = 7

In [8]:
date_time = datetime.now().strftime("%Y%m%d_%H")  # %M
save_pt_model_path = f'{project_PATH}/{step_now}/save_pt_model_s4/{date_time}'
try:
    os.mkdir(save_pt_model_path)
    print(f"Folder '{save_pt_model_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{save_pt_model_path}' already exists.")

Folder '/home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04' already exists.


In [9]:
data = np.load(PATH, allow_pickle=True) 
audio_max_value =  float((data['sounds_train']).max())*2 # the max value of the sound samples
audio_min_value =  float((data['sounds_train']).min())*2 # the max value of the sound samples
audio_max_value, audio_min_value

(3.0242209434509277, -3.1673121452331543)

In [10]:
order = [0,1,2,3,4,5,6]

def one_hot_encode(data, order):
    index_map = {number: index for index, number in enumerate(order)}
    one_hot_list = []

    for num in data:
        one_hot = [0] * len(order)
        if num in index_map:
            one_hot[index_map[num]] = 1
        one_hot_list.append(one_hot)

    return one_hot_list

## define the variables opt and set values

In [11]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNetV2',  required=False);
    parser.add_argument('--data', default='trainSet',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['2']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();
    return opt
    # opt = parser.parse_args();

## Define the Generator

In [12]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples=None, labels=None, options=None, classes_dict=None):
        random.seed(seed);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = classes_dict

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            # print(f"nClasses:{self.opt.nClasses}, type of mapdict:{type(self.mapdict)}, type of label1:{type(label1)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = label1
            idx2 = label2
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.regularization(audio_max_value, audio_min_value)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

## Define Trainer

In [13]:
class ReTrainer:
    def __init__(self, opt=None, classes_dict=None):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;
        self.trainGen = getTrainGen(opt,classes_dict=classes_dict)#train_generator.setup(opt, split);
        # self.opt.trainer = self;
        # self.trainGen = getTrainGen(self.opt, self.opt.splits)#train_generator.setup(self.opt, self.opt.split);
        # self.pretrainedmodelpath = "./resources/pretrained_models/acdnet20_20khz_fold4.h5"

    def Train(self):
        train_start_time = time.time();
        state = torch.load(second_pruned_model, map_location="cuda")
        weights = state['weight']
        config = state['config']
        print(f"config is {config}")
        net = models.GetACDNetModel(input_len=inp_len, sr=sr, nclass=self.opt.nClasses, channel_config=config)
        net.load_state_dict(weights);
         #show the acdnet structures
        calc.summary(net,(1,1,inp_len))
        # net = getPrunedModel(pruned_model_path=pruned_acdnet)
        #print networks parameters' require_grade value
        for k_, v_ in net.named_parameters():
            print(f"{k_}:{v_.requires_grad}")
        print('ACDNet model has been prepared for training');

        calc.summary(net, (1,1,self.opt.inputLength));
        # net = net.cuda();
        # training_text = "Re-Training" if self.opt.retrain else "Training from Scratch";
        # print("{} has been started. You will see update after finishing every training epoch and validation".format(training_text));

        lossFunc = torch.nn.KLDivLoss(reduction='batchmean');
        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True);

        # self.opt.nEpochs = 1957 if self.opt.split == 4 else 2000;
        for epochIdx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epochIdx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            for batchIdx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batchIdx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);
                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                # outputs = net(x);#in office and use cpu
                x = x.type(torch.FloatTensor) #use apple m2
                outputs = net(x)
                # in office use cpu, need to change to cuda
                # running_acc += (((outputs.data.argmax(dim=1) == y.argmax(dim=1))*1).float().mean()).item();
                # at home use apple m2
                res_y = y.argmax(dim=1)
                res_y = res_y.type(torch.FloatTensor)
                running_acc += ((( outputs.data.argmax(dim=1) == res_y)*1).float().mean()).item();
                y = y.type(torch.FloatTensor)
                
                loss = lossFunc(outputs.log(), y);
                loss.backward();
                optimizer.step();

                running_loss += loss.item();

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;

            net.eval();
            val_acc, val_loss = self.__validate(net, lossFunc);
            #Save best model
            # self.__save_model(val_acc, epochIdx, net);
            # ratio, acc, tr_acc, epochIdx, net
            self.__do_save_model(self.opt.pruningRatio*100, val_acc, tr_acc, epochIdx, net);
            self.__on_epoch_end(epoch_start_time, epoch_train_time, epochIdx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - train_start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));

    def load_test_data(self):
        data = np.load(self.opt.Data_npz_path, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['labels_val'])}")
        dataX = data['sounds_val'].reshape(data['sounds_val'].shape[0],1,1,data['sounds_val'].shape[1]).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(one_hot_encode(data['labels_val'], order)).type(torch.float32).to(self.opt.device);

    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __get_batch(self, index):
        x = self.trainX[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        y = self.trainY[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        return x.to(self.opt.device), y.to(self.opt.device);

    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data();
        net.eval();
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#             for idx in range(len(self.testX)):
#             x = self.testX[idx*batch_size : (idx+1)*batch_size];
            x = self.testX[:];
            x = torch.tensor(x)
            x = x.type(torch.FloatTensor) # use apple mp2
            scores = net(x);
            y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
            acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
        net.train();
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        print(f"shape of y_pred:{y_pred.shape}");
        print(f"shape of y_target:{y_target.shape}");
        
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
            if self.opt.nCrops == 1:
                y_pred = y_pred.argmax(dim=1);
                y_target = y_target.argmax(dim=1);
            else:
                y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
                y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
                print(f"after: len of y_pred:{len(y_pred)}, len of y_target:{len(y_target)}")
            y_target = y_target.cpu() #use apple m2, in office use cuda
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = lossFunc(y_pred.float().log(), y_target.float()).item();
            # loss = 0.0;
        return acc, loss;

    def __on_epoch_end(self, start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - start_time;
        val_time = epoch_time - train_time;
        line = 'SP-{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.splits, epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();

    # def __save_model(self, acc, epochIdx, net):
    #     if acc > self.bestAcc:
    #         self.bestAcc = acc;
    #         self.bestAccEpoch = epochIdx +1;
    #         save_model_name = self.opt.model_name.format(self.opt.pruningRatio*100,acc, epochIdx, genDataTimeStr());
    #         save_model_fullpath = self.opt.saveDir + save_model_name;
    #         print(f"save model to {save_model_fullpath}")
    #         torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);

    def __save_model_v2(self, acc, train_acc, epochIdx, net):
        if acc > self.bestAcc and acc > self.opt.first_save_acc:
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            self.__do_save_model(self.opt.pruningRatio*100, acc, train_acc, epochIdx, net);
        else:
            if acc > self.opt.save_val_acc and train_acc > self.opt.save_train_acc: 
                self.__do_save_model(self.opt.pruningRatio*100, acc, train_acc, epochIdx, net);
            else:
                pass

    def __do_save_model(self, ratio, acc, tr_acc, epochIdx, net):
        save_model_name = self.opt.model_name.format(ratio, acc, tr_acc, epochIdx+1, genDataTimeStr());
        save_model_fullpath = self.opt.saveDir + save_model_name;
        print(f"save model to {save_model_fullpath}")
        torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);
        # logObj.write(f"save model:{self.opt.model_name}, bestAcc:{self.bestAcc}, valAcc:{acc}, trainAcc:{tr_acc}, record@{epochIdx}-epoch");
        # logObj.write("\n");
        # logObj.flush();

In [14]:
def getTrainGen(opt=None, split=None, classes_dict=None):
    
    dataset = np.load(opt.Data_npz_path, allow_pickle=True);
    train_sounds = []
    train_labels = []

    train_sounds = dataset['sounds_train']
    train_labels = dataset['labels_train']

    trainGen = TLGenerator(train_sounds, train_labels, opt, classes_dict=classes_dict);
    trainGen.preprocess_setup();
    return trainGen

In [15]:
def main():
    map_dict_train = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7,}
    
    opt = getOpts();
    opt.pruningRatio = 0.9
    save_dir = "{}/retrain_4C_after_2nd_pruning_prunratio{}_{}/".format(save_pt_model_path,opt.pruningRatio,getDateStr());
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    print(f"save dir is: {save_dir}");

    opt.Data_npz_path = PATH
    opt.batchSize = 32;
    opt.LR = 0.05;
    opt.weightDecay = 5e-4;
    opt.momentum = 0.09;
    opt.nEpochs = 2000;
    opt.schedule = [0.1,0.3, 0.6, 0.9];
    opt.warmup = 10;
    
    opt.saveDir = save_dir;
    # opt.device="mps";
    if torch.backends.mps.is_available():
        opt.device="mps"; #for apple m2 gpu
    elif torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = ch_n_class
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = sr;
    opt.inputLength = inp_len;
    #Test data
    opt.nCrops = 2;
    opt.model_name = "retrained_4Cmodel_ratio{}_vaacc{}_tracc_{}@{}epoch_{}.pt"
    #Starting retraining process
    trainer = ReTrainer(opt=opt, classes_dict=map_dict_train);
    trainer.Train();

In [16]:
main()

save dir is: /home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/
***Use device:cuda:0


length of samples:1018
config is [8, 40, 23, 51, 57, 81, 90, 89, 7]
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 20150)     (8, 1, 10071)         72      725,112
  BatchNorm2d-2     (8, 1, 10071)     (8, 1, 10071)         16            0
         ReLu-3     (8, 1, 10071)     (8, 1, 10071)          0       80,568
       Conv2d-4     (8, 1, 10071)     (40, 1, 5034)      1,600    8,054,400
  BatchNorm2d-5     (40, 1, 5034)     (40, 1, 5034)         80            0
         ReLu-6     (40, 1, 5034)     (40, 1, 5034)          0      201,360
    MaxPool2d-7     (40, 1, 5034)      (40, 1, 100)          0      200,000
      Permute-8      (40, 1, 100)      (1, 40, 100)          0            0
       Conv

/tmp/ipykernel_96925/2409876903.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(second_pruned_model, map_location="cuda")


device is :cuda:0
len of Y:100
shape of y_pred:torch.Size([100, 7])
shape of y_target:torch.Size([100, 7])
after: len of y_pred:50, len of y_target:50
save model to /home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc56.0_tracc_21.875@1epoch_20240901045049.pt
SP-[1] Epoch: 1/2000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.72  Acc 21.88% | Val: Loss nan  Acc(top1) 56.00% | HA 0.00@0


/tmp/ipykernel_96925/2409876903.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)


shape of y_pred:torch.Size([100, 7])
shape of y_target:torch.Size([100, 7])
after: len of y_pred:50, len of y_target:50
save model to /home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc62.0_tracc_36.62109375@2epoch_20240901045057.pt
SP-[1] Epoch: 2/2000 | Time: 0m08s (Train 0m08s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.25  Acc 36.62% | Val: Loss nan  Acc(top1) 62.00% | HA 0.00@0
shape of y_pred:torch.Size([100, 7])
shape of y_target:torch.Size([100, 7])
after: len of y_pred:50, len of y_target:50
save model to /home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc58.0_tracc_45.3125@3epoch_20240901045105.pt
SP-[1] Epoch: 3/2000 | Time: 0m07s (Train 0m07s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.15  Acc 45.31% | Val: Loss nan  Acc(top1) 58.00% 